In [ ]:
import pandas as pd
import numpy as np

In [ ]:
event_log = pd.read_csv('./data/dossiers v2 - event log.csv').dropna()
event_log

In [ ]:
event_log['action'] = event_log['cat_actie_omschrijving'] + '-' + event_log['actie_omschrijving']

In [ ]:
event_log[event_log['bedrag_betaling'] != 0]['dossier_nr'].nunique()

## Filtering on actions

In [ ]:
important_actions = pd.read_excel('./data/important actions.xlsx')
important_actions['action'] = important_actions['cat_actie_omschrijving'] + '-' + important_actions['actie_omschrijving']
important_actions

Replace a wrong value

In [ ]:
event_log['action'] = event_log['action'].str.replace('?', '–', regex=False) 

In [ ]:
important_actions[~important_actions['action'].isin(event_log['action'])]

In [ ]:
# event_log = event_log[event_log['action'].isin(important_actions['action'])]

In [ ]:
event_log[(event_log['actie_omschrijving'] == 'deb betaling fortis i-deal') & (event_log.cat_actie_code == 999.0)]

In [ ]:
important_actions.cat_actie_code.value_counts()

In [ ]:
event_log2 = event_log[~event_log['action'].isin(important_actions['action'])]
event_log2[event_log2['bedrag_betaling'] != 0]['actie_omschrijving'].value_counts()

In [ ]:
event_log = event_log[event_log['action'].isin(important_actions['action'])]

In [ ]:
event_log['action'].nunique()

In [ ]:
event_log.loc[(event_log.action == '[310] inkomende_betaling_derde-ABNbankbetaling via werkg/uitk') & (event_log.bedrag_betaling == 0)]

In [ ]:
event_log[event_log['bedrag_betaling'] != 0]['dossier_nr'].nunique()

## Aggregating payments

In [ ]:
event_log[event_log.cat_actie_code == 110]['actie_omschrijving'].value_counts()

In [ ]:
event_log[event_log.cat_actie_code == 210]['actie_omschrijving'].value_counts()

In [ ]:
event_log.loc[event_log.bedrag_betaling > 0].cat_actie_code.value_counts()

In [ ]:
event_log.loc[event_log.actie_omschrijving == 'Inkomende post opdrachtgever: betalingsmelding']

In [ ]:
event_log.loc[event_log.actie_omschrijving == 'Inkomende post opdrachtgever: betalingsmelding', 'action'] = 'payment'

In [ ]:
# event_log.loc[event_log.action == '[310] inkomende_betaling_derde-ABNbankbetaling via werkg/uitk', 'action'] = 'payment'

In [ ]:
event_log.loc[event_log.bedrag_betaling > 0, 'action'] = 'payment'

In [ ]:
event_log.loc[(event_log.bedrag_betaling == 0) & (event_log.cat_actie_code == 210), 'action'] = 'unsuccesful payment'

In [ ]:
event_log[event_log['action'] == 'payment']

In [ ]:
event_log['action'].nunique()

In [ ]:
event_log[event_log['bedrag_betaling'] != 0]['dossier_nr'].nunique()

## Filtering on case coverage

In [ ]:
case_coverage = [{'activity' : activity, 'nr_of_cases' : event_log[event_log['action'] == activity]['dossier_nr'].nunique()} for activity in event_log['action'].unique()]

In [ ]:
event_log['action'].nunique()

In [ ]:
case_coverage_df = pd.DataFrame(case_coverage)
case_coverage_df

In [ ]:
important_actions[~important_actions['action'].isin(case_coverage_df['activity'])]

In [ ]:
event_log['dossier_nr'].nunique() * 0.05

In [ ]:
len(case_coverage_df)

In [ ]:
case_coverage_df

In [ ]:
case_coverage_df = case_coverage_df[case_coverage_df.nr_of_cases > event_log['dossier_nr'].nunique() * 0.05]

In [ ]:
event_log = event_log[event_log['action'].isin(case_coverage_df['activity'])]

In [ ]:
event_log

In [ ]:
event_log[event_log['action'] == 'payment']['dossier_nr'].nunique()

## Cutting all activities after payment

In [ ]:
event_log['action'].value_counts()[0]

In [ ]:
multi_index = event_log.set_index(['dossier_nr', 'volgnr_actie'])

In [ ]:
# for (i, j), data in multi_index.iterrows():
res = pd.DataFrame()
for dossier_nr, data in multi_index.groupby(level=0):
    first_payment = None
#     print(data)
    if (data.action == 'payment').any():
        first_payment = (data.action == 'payment').argmax() + 1 #TODo remove voor final dataset
#         print(data.iloc[:first_payment])
        res = res.append(data.iloc[:first_payment])
    else:
        res = res.append(data)
res
#     print(first_payment)
#     print(data.iloc[first_payment])

In [ ]:
res.to_csv('./data/BPM_event_log_cutoff.csv')

In [ ]:
dataatje = res.copy()

In [ ]:
dataatje.reset_index()

In [ ]:
event_log.to_csv('./data/BPM_event_log.csv')